In [2]:
import os
import tensorflow as tf
import import_ipynb

In [8]:
from datetime import datetime
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers, Sequential
from data_preprocessing import train_ds, test_ds
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard, EarlyStopping

In [5]:
num_classes = 7

In [6]:
model = Sequential([
    layers.Conv2D(32, (3, 3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(64, (3, 3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(128, (3, 3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),
    
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.2),
    
    layers.Dense(num_classes, activation="softmax")
])

In [10]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [11]:
model.compile(optimizer=sgd, loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

In [ ]:
model.fit(train_ds, epochs=10,
         use_multiprocessing=True,
         workers=5,
         max_queue_size=5)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
718/718 [==============================] - 12s 17ms/step - loss: 2.1498 - accuracy: 0.0157
Epoch 2/10
718/718 [==============================] - 6s 9ms/step - loss: 2.1502 - accuracy: 0.0154
Epoch 3/10
718/718 [==============================] - 6s 9ms/step - loss: 2.1502 - accuracy: 0.0154: 5s - loss: 2.1530 - accura - ETA: 5s - loss: 2.1531 - accura - ETA: 4s - lo